<a href="https://colab.research.google.com/github/choiiiiii/stan/blob/master/20Stan_golf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
from scipy.stats import beta
import matplotlib.pyplot as plt
import os
from cmdstanpy import CmdStanModel, cmdstan_path

In [12]:
df = pd.read_csv('data/golf_data.txt')
new_df = pd.read_csv('data/golf_newdata.txt')

# Model building and expansion for golf putting


Andrew Gelman


24 Sep 2019


The following graph shows data from professional golfers on the proportion of successful putts as a function of distance from the hole. Unsurprisingly, the probability of making the shot declines as a function of distance:

![title](https://github.com/choiiiiii/stan/blob/master/data/img/golf_data.png?raw=1)


The error bars associated with each point $j$
in the above graph are simple classical standard deviations, $ \sqrt{\hat{p_j}(1- \hat{p_j}/ n_j)} $, where $\hat{p_j} = y_j/n_j$ is the success rate for putts taken at distance $x_j$

## Logistic regression
Can we model the probability of success in golf putting as a function of distance from the hole? Given usual statistical practice, the natural starting point would be logistic regression:

$$y_j \sim binomial(n_j,logit^{-1}(a+bx_j)),\ for\ j=1,\dots\, J$$

In Stan, this is:

```
data {
  int J;
  int n[J];
  vector[J] x;
  int y[J];
}
parameters {
  real a;
  real b;
}
model {
  y ~ binomial_logit(n, a + b*x);
}
```

The code in the above model block is (implicitly) vectorized, so that it is mathematically equivalent to modeling each data point, `y[i] ~ binomial_logit(n[i],a+b*x[i])`. The vectorized code is more compact (no need to write a loop, or to include the subscripts) and faster (because of more efficient gradient evaluations).


We fit the model to the data:

In [3]:
golf_logistic_path = os.path.join('./stanfile','golf_logistic.stan')
golf_logistic_model = CmdStanModel(stan_file=golf_logistic_path)
golf_logistic_data = {
    "J": df.shape[0],
    "n": list(df.loc[:,'n']),
    "x": list(df.loc[:,'x']),
    "y": list(df.loc[:,'y'])
}
golf_logistic_fit = golf_logistic_model.sample(chains=5, cores=3, data=golf_logistic_data)

INFO:cmdstanpy:compiling stan program, exe file: /Users/hyunjimoon/Dropbox/stan/casestudy/stanfile/golf_logistic
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /Users/hyunjimoon/Dropbox/stan/casestudy/stanfile/golf_logistic
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 5
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:finish chain 5


And here is the result:

In [4]:
golf_logistic_fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-3021.15000,0.025314,0.976643,-3023.060000,-3020.830000,-3020.210000,1488.49,7006.78,1.00362
a,2.23158,0.001646,0.058184,2.133080,2.232660,2.324820,1249.58,5882.18,1.00527
b,-0.25580,0.000181,0.006614,-0.266226,-0.255982,-0.244599,1329.12,6256.61,1.00434


Going through the columns of the above table: Stan has computed the posterior means ± standard deviations of $a$ and $b$ to be 2.23 ± 0.06 and -0.26 ± 0.01, respectively. The Monte Carlo standard error of the mean of each of these parameters is 0 (to two decimal places), indicating that the simulations have run long enough to estimate the posterior means precisely. The posterior quantiles give a sense of the uncertainty in the parameters, with 50% posterior intervals of $[2.19, 2.27]$ and $[-0.26, -0.25]$ for $a$ and $b$, respectively. Finally, the values of $\hat{R}$ near 1 tell us that the simulations from Stan’s four simulated chains have mixed well. (We have more sophisticated convergence diagnostics, and also we recommend checking the fit using simulated data, as discussed in the Bayesian Workflow Using Stan book, but checking that $\hat{R}$
is near 1 is a good start.)
The following graph shows the fit plotted along with the data:

![title](https://github.com/choiiiiii/stan/blob/master/data/img/logistic_regression.png?raw=1)

The black line shows the fit corresponding to the posterior median estimates of the parameters $a$ and $b$; the green lines show 10 draws from the posterior distribution.


In this example, posterior uncertainties in the fits are small, and for simplicity we will just plot point estimates based on posterior median parameter estimates for the remaining models. Our focus here is on the sequence of models that we fit, not so much on uncertainty in particular model fits.

## Modeling from first principles

As an alternative to logistic regression, we shall build a model from first principles and fit it to the data.

The graph below shows a simplified sketch of a golf shot. The dotted line represents the angle within which the ball of radius $r$ must be hit so that it falls within the hole of radius $R$. This threshold angle is $sin^{-1}((R-r)/x)$. The graph, which is not to scale, is intended to illustrate the geometry of the ball needing to go into the hole.

![title](https://github.com/choiiiiii/stan/blob/master/data/img/golf_geometry.png?raw=1)

The next step is to model human error. We assume that the golfer is attempting to hit the ball completely straight but that many small factors interfere with this goal, so that the actual angle follows a normal distribution centered at 0 with some standard deviation $\sigma$.

![title](https://github.com/choiiiiii/stan/blob/master/data/img/angle_of_shot.png?raw=1)

The probability the ball goes in the hole is then the probability that the angle is less than the threshold; that is, 
$Pr(|angle| < sin^{-1}((R-r)/x) = 2\Phi(\frac{sin^{-1}(R-r)/x)}{\sigma})-1$,where $\Phi$ is the cumulative normal distribution function. The only unknown parameter in this model is $\sigma$, the standard deviation of the distribution of shot angles. Stan (and, for that matter, R) computes trigonometry using angles in radians, so at the end of our calculations we will need to multiply by $180/\pi$
to convert to degrees, which are more interpretable by humans.


Our model then has two parts: 

$$y_j \sim binomial(n_j,p_j)$$
$$p_j = 2\Phi(\frac{sin^{-1}(R-r)/x)}{\sigma}) - 1,\ for\ j=1,\dots,J$$

Here is a graph showing the curve for some potential values of the parameter $\sigma$.


![title](https://github.com/choiiiiii/stan/blob/master/data/img/model_comparison_sigma.png?raw=1)

The highest curve on the graph corresponds to $\sigma = 0.5^{\circ}$: if golfers could control the angles of their putts to an accuracy of approximately half a degree, they would have a very high probability of success, making over 80% of their ten-foot putts, over 50% of their fifteen-foot putts, and so on. At the other extreme, the lowest plotted curve corresponds to $\sigma = 20^{\circ}$: if your putts could be off as high as 20 degrees, then you would be highly inaccurate, missing more than half of your two-foot putts. When fitting the model in Stan, the program moves around the space of $\sigma$, sampling from the posterior distribution.

We now write the Stan model in preparation to estimating $\sigma$ :



```
data {
  int J;
  int n[J];
  vector[J] x;
  int y[J];
  real r;
  real R;
}
transformed data {
  vector[J] threshold_angle = asin((R-r) ./ x);
}
parameters {
  real<lower=0> sigma;
}
model {
  vector[J] p = 2*Phi(threshold_angle / sigma) - 1;
  y ~ binomial(n, p);
}
generated quantities {
  real sigma_degrees = sigma * 180 / pi();
}
```




In the transformed data block above, the `./` in the calculation of p corresponds to componentwise division in this vectorized computation.



The data $J,n,x,y$ have already been set up; we just need to define $r$ and $R$ (the golf ball and hole have diameters 1.68 and 4.25 inches, respectively), and run the Stan model:

In [5]:
golf_angle_path = os.path.join('./stanfile','golf_angle.stan')
golf_angle_model = CmdStanModel(stan_file=golf_angle_path)
golf_angle_data = {
    "J": df.shape[0],
    "n": list(df.loc[:,'n']),
    "x": list(df.loc[:,'x']),
    "y": list(df.loc[:,'y']),
    "r": (1.68/2)/12,
    "R": (4.25/2)/12,
}
golf_angle_fit = golf_angle_model.sample(chains=5, cores=3, data=golf_angle_data)

INFO:cmdstanpy:compiling stan program, exe file: /Users/hyunjimoon/Dropbox/stan/casestudy/stanfile/golf_angle
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /Users/hyunjimoon/Dropbox/stan/casestudy/stanfile/golf_angle
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:start chain 5
INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:finish chain 5


Here is the result:

In [6]:
golf_angle_fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-2926.800000,0.015990,0.744027,-2928.340000,-2926.51000,-2926.270000,2165.01,10092.60,1.00194
sigma,0.026662,0.000009,0.000411,0.025998,0.02666,0.027338,1928.90,8991.92,1.00066
sigma_degrees,1.527620,0.000536,0.023526,1.489570,1.52750,1.566360,1928.90,8991.89,1.00066


The model has a single parameter, $\sigma$. From the output, we find that Stan has computed the posterior mean of $\sigma$ to be 0.03. Multiplying this by $180/\pi$
, this comes to 1.53 degrees. The Monte Carlo standard error of the mean is 0 (to two decimal places), indicating that the simulations have run long enough to estimate the posterior mean precisely. The posterior standard deviation is calculated at 0.02 degrees, indicating that $\sigma$ itself has been estimated with high precision, which makes sense given the large number of data points and the simplicity of the model. The precise posterior distribution of $\sigma$ can also be seen from the narrow range of the posterior quantiles. Finally, $\hat{R}$ is near 1, telling us that the simulations from Stan’s four simulated chains have mixed well.

We next plot the data and the fitted model (here using the posterior median of $\sigma$
but in this case the uncertainty is so narrow that any reasonable posterior summary would give essentially the same result), along with the logistic regression fitted earlier:

![title](https://github.com/choiiiiii/stan/blob/master/data/img/model_comparison.png?raw=1)

The custom nonlinear model fits the data much better. This is not to say that the model is perfect—any experience of golf will reveal that the angle is not the only factor determining whether the ball goes in the hole—but it seems like a useful start, and it is good to know that we can fit nonlinear models by just coding them up in Stan.


## Testing the fitted model on new data

Recently a local business school professor and golfer, Mark Broadie, came by my office with tons of new data. For simplicity we’ll just look here at the summary data, probabilities of the ball going into the hole for shots up to 75 feet from the hole. The graph below shows these new data (in red), along with our earlier dataset (in blue) and the already-fit geometry-based model from before, extending to the range of the new data.



![title](https://github.com/choiiiiii/stan/blob/master/data/img/checking_alreadyfit.png?raw=1)


Comparing the two datasets in the range 0-20 feet, the success rate is similar for longer putts but is much higher than before for the short putts. This could be a measurement issue, if the distances to the hole are only approximate for the old data, and it could also be that golfers are better than they used to be.

Beyond 20 feet, the empirical success rates become lower than would be predicted by the old model. These are much more difficult attempts, even after accounting for the increased angular precision required as distance goes up.

## A new model accounting for how hard the ball is hit
To get the ball in the hole, the angle isn’t the only thing you need to control; you also need to hit the ball just hard enough.

Mark Broadie added this to our model by introducing another parameter corresponding to the golfer’s control over distance. Supposing uu is the distance that golfer’s shot would travel if there were no hole, Broadie assumes that the putt will go in if (a) the angle allows the ball to go over the hole, and (b) uu is in the range [x,x+3]. That is the ball must be hit hard enough to reach the whole but not go too far. Factor (a) is what we have considered earlier; we must now add factor (b).

The following sketch, which is not to scale, illustrates the need for the distance as angle as well as the angle of the shot to be in some range, in this case the gray zone which represents the trajectories for which the ball would reach the hole and stay in it.

![title](https://github.com/choiiiiii/stan/blob/master/data/img/golf_physical_model.png?raw=1)

Broadie supposes that a golfer will aim to hit the ball one foot past the hole but with a multiplicative error in the shot’s potential distance, so that u=(x+1)⋅(1+error)u=(x+1)⋅(1+error), where the error has a normal distribution with mean 0 and standard deviation σdistanceσdistance. This new parameter σdistanceσdistance represents the uncertainty in the shot’s relative distance. In statistics notation, this model is,

$$ u∼normal(x+1,(x+1)\sigma_{distance }),$$
and the distance is acceptable if u∈[x,x+3], an event that has probability 
$$\Phi\left(\frac{2}{(x+1) \sigma_{distance}}\right)-\Phi\left(\frac{-1}{(x+1) \sigma_{distance}}\right)$$

Putting these together, the probability a shot goes in becomes, 

$$\left(2 \Phi\left(\frac{\sin ^{-1}((R-r)(x)}{\sigma_{\text {angle}}}\right)-1\right)\left(\Phi\left(\frac{2}{(x+1) \sigma_{\text {distance }}}\right)-\Phi\left(\frac{-1}{(x+1) \sigma_{\text {distance }}}\right)\right)$$

where we have renamed the parameter σσ from our earlier model to σangleσangle to distinguish it from the new σdistanceσdistance parameter. We write the new model in Stan, giving it the name `golf_angle_distance_2.stan` to convey that it is the second model in a series, and that it accounts both for angle and distance:

```
data {
  int J;
  int n[J];
  vector[J] x;
  int y[J];
  real r;
  real R;
  real overshot;
  real distance_tolerance;
}
transformed data {
  vector[J] threshold_angle = asin((R-r) ./ x);
}
parameters {
  real<lower=0> sigma_angle;
  real<lower=0> sigma_distance;
}
model {
  vector[J] p_angle = 2*Phi(threshold_angle / sigma_angle) - 1;
  vector[J] p_distance = Phi((distance_tolerance - overshot) ./ ((x + overshot)*sigma_distance)) -
               Phi((- overshot) ./ ((x + overshot)*sigma_distance));
  vector[J] p = p_angle .* p_distance;
  y ~ binomial(n, p);
  [sigma_angle, sigma_distance] ~ normal(0, 1);
}
generated quantities {
  real sigma_degrees = sigma_angle * 180 / pi();
}
```

Here we have defined `overshot` and `distance_tolerance` as data, which Broadie has specified as 1 and 3 feet, respectively. We might wonder why if the distance range is 3 feet, the overshot is not 1.5 feet. One reason could be that it is riskier to hit the ball too hard than too soft. In addition we assigned weakly informative half-normal(0,1) priors on the scale parameters, $\sigma_{angle}$ and $\sigma_{distance}$, which are required in this case to keep the computations stable.

In [7]:
golf_angle_distance_2_path = os.path.join('./stanfile','golf_angle_distance_2.stan')
golf_angle_distance_2_model = CmdStanModel(stan_file=golf_angle_distance_2_path)
golf_angle_distance_2_data = {
    "J": new_df.shape[0],
    "n": list(new_df.loc[:,'n']),
    "x": list(new_df.loc[:,'x']),
    "y": list(new_df.loc[:,'y']),
    "r": (1.68/2)/12,
    "R": (4.25/2)/12,
    "overshot": 1,
    "distance_tolerance": 3,
}
golf_angle_distance_2_fit = golf_angle_distance_2_model.sample(chains=5, cores=3, data=golf_angle_distance_2_data)

INFO:cmdstanpy:compiling stan program, exe file: /Users/hyunjimoon/Dropbox/stan/casestudy/stanfile/golf_angle_distance_2
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /Users/hyunjimoon/Dropbox/stan/casestudy/stanfile/golf_angle_distance_2
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 5
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 5
INFO:cmdstanpy:finish chain 4


## Fitting the new model to data
We fit the model to the new dataset.

![title](https://github.com/choiiiiii/stan/blob/master/data/img/errormessage.png?raw=1)

There is poor convergence, and we need to figure out what is going on here. (Problems with computation often indicate underlying problems with the model being fit. That’s the folk theorem of statistical computing.)

In [8]:
golf_angle_distance_2_fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-407982.000000,53989.100000,85493.000000,-578947.000000,-364911.000000,-364909.000000,2.50754,0.093433,364.2760
sigma_angle,0.021071,0.008573,0.013578,0.013202,0.013350,0.048027,2.50833,0.093462,62.5012
sigma_distance,0.116983,0.015978,0.025304,0.078729,0.136690,0.137993,2.50817,0.093457,60.7203
sigma_degrees,1.207270,0.491222,0.777982,0.756430,0.764907,2.751740,2.50833,0.093462,62.5010


To understand what is happening, we graph the new data and the fitted model, accepting that this “fit,” based as it is on poorly-mixing chains, is only provisional:



![title](https://github.com/choiiiiii/stan/blob/master/data/img/checking_modelfit.png?raw=1)

To understand what is happening, we graph the new data and the fitted model, accepting that this “fit,” based as it is on poorly-mixing chains, is only provisional:

In [9]:
new_df.iloc[:5,]

,x,n,y
0,0.28,45198,45183
1,0.97,183020,182899
2,1.93,169503,168594
3,2.92,113094,108953
4,3.93,73855,64740


With such large values of $n_j$, the binomial likelihood enforces an extremely close fit at these first few points, and that drives the entire fit of the model.

To fix this problem we took the data model, $y_j∼binomial(n_j,p_j)$, and added an independent error term to each observation. There is no easy way to add error directly to the binomial distribution—we could replace it with its overdispersed generalization, the beta-binomial, but this would not be appropriate here because the variance for each data point ii would still be roughly proportional to the sample size njnj, and our whole point here is to get away from this assumption and allow for model misspecification—so instead we first approximate the binomial data distribution by a normal and then add independent variance; thus:

$$
y_{j} / n_{j} \sim \text { normal }\left(p_{j}, \sqrt{p_{j}\left(1-p_{j}\right) / n_{j}+\sigma_{y}^{2}}\right)
$$

To write this in Stan there are some complications:

- y and n are integer variables, which we convert to vectors so that we can multiply and divide them.

- To perform componentwise multiplication or division using vectors, you need to use `.*` or `./` so that San knows not to try to perform vector/matrix multiplication and division. Stan is opposite from R in this way: Stan defaults to vector/matrix operations and has to be told otherwise, whereas R defaults to componentwise operations, and vector/matrix multiplication in R is indicated using the `%*%` operator.

We implement these via the following new code in the transformed data block:
```
vector[J] raw_proportions = to_vector(y) ./ to_vector(n);
```
And then in the model block:
```
  raw_proportions ~ normal(p, sqrt(p .* (1-p) ./ to_vector(n) + sigma_y^2));
```

To complete the model we add $\sigma_y$ to the parameters block and assign it a weakly informative half-normal(0,1) prior distribution. Here’s the new Stan program:
```
data {
  int J;
  int n[J];
  vector[J] x;
  int y[J];
  real r;
  real R;
  real overshot;
  real distance_tolerance;
}
transformed data {
  vector[J] threshold_angle = asin((R-r) ./ x);
  vector[J] raw_proportions = to_vector(y) ./ to_vector(n);
}
parameters {
  real<lower=0> sigma_angle;
  real<lower=0> sigma_distance;
  real<lower=0> sigma_y;
}
model {
  vector[J] p_angle = 2*Phi(threshold_angle / sigma_angle) - 1;
  vector[J] p_distance = Phi((distance_tolerance - overshot) ./ ((x + overshot)*sigma_distance)) -
               Phi((- overshot) ./ ((x + overshot)*sigma_distance));
  vector[J] p = p_angle .* p_distance;
  raw_proportions ~ normal(p, sqrt(p .* (1-p) ./ to_vector(n) + sigma_y^2));
  [sigma_angle, sigma_distance, sigma_y] ~ normal(0, 1);
}
generated quantities {
  real sigma_degrees = sigma_angle * 180 / pi();
}
```

We now fit this model to the data:

In [10]:
golf_angle_distance_3_path = os.path.join('./stanfile','golf_angle_distance_3.stan')
golf_angle_distance_3_model = CmdStanModel(stan_file=golf_angle_distance_3_path)
golf_angle_distance_3_data = {
    "J": new_df.shape[0],
    "n": list(new_df.loc[:,'n']),
    "x": list(new_df.loc[:,'x']),
    "y": list(new_df.loc[:,'y']),
    "r": (1.68/2)/12,
    "R": (4.25/2)/12,
    "overshot": 1,
    "distance_tolerance": 3,
}
golf_angle_distance_3_fit = golf_angle_distance_3_model.sample(chains=5, cores=3, data=golf_angle_distance_3_data)
golf_angle_distance_3_fit.summary()


INFO:cmdstanpy:compiling stan program, exe file: /Users/hyunjimoon/Dropbox/stan/casestudy/stanfile/golf_angle_distance_3
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /Users/hyunjimoon/Dropbox/stan/casestudy/stanfile/golf_angle_distance_3
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:start chain 5
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:finish chain 5


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,146.552000,0.029646,1.290160,144.013000,146.875000,147.958000,1893.92,1767.16,1.00103
sigma_angle,0.017807,0.000002,0.000104,0.017638,0.017806,0.017979,2009.62,1875.11,1.00113
sigma_distance,0.080030,0.000029,0.001297,0.077901,0.080040,0.082170,1988.39,1855.30,1.00115
sigma_y,0.003051,0.000012,0.000593,0.002186,0.002997,0.004091,2363.75,2205.53,1.00124
sigma_degrees,1.020260,0.000133,0.005956,1.010610,1.020230,1.030130,2009.58,1875.07,1.00113


The new parameter estimates are:

- $\sigma_{angle}$ is estimated at 0.02, which when corresponds to $\sigma_{degrees}$  = 1.0. According to the fitted model, there is a standard deviation of 1.0 degree in the angles of putts taken by pro golfers. The estimate of $\sigma_{angle}$ has decreased compared to the earlier model that only had angular errors. This makes sense: now that distance errors have been included in the model, there is no need to explain so many of the missed shots using errors in angle.

- $\sigma_{distance}$  is estimated at 0.08. According to the fitted model, there is a standard deviation of 8% in the errors of distance.

- $\sigma_{y}$ is estimated at 0.003. The fitted model fits the aggregate data (success rate as a function of distance) to an accuracy of 0.3 percentage points.

And now we graph:

![title](https://github.com/choiiiiii/stan/blob/master/data/img/checking_modelfit_2.png?raw=1)

We can go further and plot the residuals from this fit. First we augment the Stan model to compute residuals in the generated quantities block.

Then we compute the posterior means of the residuals, $y_{j}/n_{j}−p_{j}$, then plot these vs. distance:

![title](https://github.com/choiiiiii/stan/blob/master/data/img/residuals.png?raw=1)

The residuals are small (see the scale of the yy-axis) and show no clear pattern, suggesting not that the model is perfect but that there are no clear ways to develop it further just given the current data.

## Problems with the model and potential improvements
The error term in the above model is a hack. It’s useful to allow the model not to fit the data perfectly, but it can’t be right to model these systematic errors as being independent. In this case, though, it doesn’t really matter, given how tiny these errors are: their estimated standard deviation is less than one percentage point.

The model has two parameters that were fixed as data: `distance_tolerance`, which was set to 3 (implying that the ball will only fall into the hole if it is hit on a trajectory that would go past the hole, but no more than 3 feet past) and `overshot`, which was set to 1 (implying that the golfer will aim 1 foot past the hole). In theory it would be possible to estimate either or both these parameters from the data. In practice, no way. The model already fits the data so well (as shown by the above graph) that there’s clearly no more information available to estimate any additional parameters. If we were to do so, the estimates would be highly noisy and unstable (if their prior is weak) or highly dependent on the prior (if an informative prior distribution is specified). Either way we don’t see the advantage of this sort of fit.

Just for laughs, though, we constructed such a model and fit it, just to see what would happen. We simply took our previous Stan program and moved these two parameters from the data block to the parameters block along with zero-boundary constraints:
```
real<lower=0> overshot;
real<lower=0> distance_tolerance;
```
And then in the model block we added weak priors centered at Broadie’s guesses and with wide uncertainties:

```  
overshot ~ normal(1, 5);
distance_tolerance ~ normal(3, 5);
  ```
Fitting this model to the data yields poor convergence and no real gain beyond the simpler version already fit in which overshot and distance_tolerance were set to fixed values.

The model is unrealistic in other ways, for example by assuming distance error is strictly proportional to distance aimed, and assuming independence of angular and distance errors. Presumably, angular error is higher for longer putts. Again, though, we can’t really investigate such things well using these data which are already such a good fit to the simple two-parameter model we have already fit.

Players vary in ability and golf courses vary in difficulty. Given more granular data, we should be able to fit a multilevel model allowing parameters to vary by player, golf course, and weather conditions.

# Summary of fitted models
We have two datasets, `golf` and `golf_new`, to which we fit several Stan models. First we fit `golf_logistic` and `golf_angle` to the `golf` dataset, then we fit `golf_angle` to the `golf_new` dataset and see a problem, then we fit `golf_angle_distance_2` and `golf_angle_distance_3` to `golf_new` and finally obtained a good fit, then we fit `golf_angle_distance_3_with_resids` which was the same model but also saving residuals. Finally, we fit `golf_angle_distance_4` to `golf_new` but we didn’t display the fit, we just discussed it.

# References
Don Berry (1995). Statistics: A Bayesian Perspective. Duxbury Press. The original golf dataset appears as an example in this book.

Mark Broadie (2018). Two simple putting models in golf. Linked from https://statmodeling.stat.columbia.edu/2019/03/21/new-golf-putting-data-and-a-new-golf-putting-model/. Here is the larger dataset and a document describing the model with angular and distance errors.

Andrew Gelman and Deborah Nolan (2002). A probability model for golf putting. Teaching Statistics 50, 151-153. Our first explanation of the angular-error model.

All code in this document is licensed under BSD 3-clause license and all text licensed under CC-BY-NC 4.0